In [1]:
import numpy as np
import h5py

import sys
sys.path.append('../src')
from hdf_dataset import HDF3DIterator

In [3]:
with h5py.File('/grid/hackathon/data_norepl/splarseers/output/embed_val.h5', 'r') as f:
    print(f['embed_val'].shape)

with h5py.File('/grid/hackathon/data_norepl/splarseers/output/embed_train.h5', 'r') as f:
    print(f['embed_train'].shape)

with h5py.File('/grid/hackathon/data_norepl/splarseers/output/embed_test.h5', 'r') as f:
    print(f['embed_test'].shape)


(58401, 5000, 32)
(210308, 5000, 32)
(12936, 5000, 32)


In [12]:
for multiplier in [1, 8, 32, 64, 256]:
    batch_number = (58401*5000)/(16384*multiplier)
    print(f'{multiplier}: batch number {batch_number}\n\t'
          f'batch size {(16384*multiplier)}\n\t'
          f'batches to dead {(600)/multiplier}\n\t'
          f'Gb/batch {(35)/(batch_number)}')


1: batch number 17822.57080078125
	batch size 16384
	batches to dead 600.0
	Gb/batch 0.001963801989691957
8: batch number 2227.8213500976562
	batch size 131072
	batches to dead 75.0
	Gb/batch 0.015710415917535657
32: batch number 556.9553375244141
	batch size 524288
	batches to dead 18.75
	Gb/batch 0.06284166367014263
64: batch number 278.47766876220703
	batch size 1048576
	batches to dead 9.375
	Gb/batch 0.12568332734028526
256: batch number 69.61941719055176
	batch size 4194304
	batches to dead 2.34375
	Gb/batch 0.502733309361141


In [6]:
!ls -lh /grid/hackathon/data_norepl/splarseers/output/

total 169G
-rw-rwx--- 1 zhliu koohackathon 7.8G Mar 13 17:02 embed_test.h5
-rw-rwx--- 1 zhliu koohackathon 126G Mar 13 17:13 embed_train.h5
-rw-rwx--- 1 zhliu koohackathon  35G Mar 13 17:04 embed_val.h5
-rw-rwx--- 1 zhliu koohackathon  12K Mar 13 18:23 test_shape.ipynb


In [9]:
fake_training_data = np.random.randn(5, 5000, 32)
with h5py.File('./fake_embed_train.h5', 'w') as f:
    f.create_dataset('embed_train', data=fake_training_data)

fake_val_data = np.random.randn(5, 5000, 32)
with h5py.File('./fake_embed_val.h5', 'w') as f:
    f.create_dataset('embed_val', data=fake_val_data)

fake_test_data = np.random.randn(5, 5000, 32)
with h5py.File('./fake_embed_test.h5', 'w') as f:
    f.create_dataset('embed_test', data=fake_test_data)



In [3]:
ds = HDF3DIterator('./fake_embed_test.h5', 'embed_test', preload=True)

with ds:
    print(ds[0])

ds.data.shape

[-0.19399936  0.65435621 -0.96320109  0.5455392  -0.04167439 -0.3720585
  0.48796122  0.30532098  0.26174639  0.23235679  1.8282784  -1.61954305
  1.64801566  0.84321343 -1.03615545 -0.51377551  1.07920278 -0.14377821
 -2.08454498 -0.72174054  0.0141697  -2.13751916 -0.86524223  1.23499815
  0.1319807  -2.35447913 -0.91882274 -0.70373064 -0.093258    1.50630295
  1.02012788  0.87943806]


AttributeError: 'NoneType' object has no attribute 'shape'

In [4]:
ds.open()
ds.data.shape


(5, 5000, 32)

In [5]:
ds = HDF3DIterator('/grid/hackathon/data_norepl/splarseers/output/embed_test.h5', 'embed_test')
ds[0]

array([9.9999976e-01, 1.5559001e-08, 2.2726178e-07], dtype=float32)

In [6]:
ds = HDF3DIterator('/grid/hackathon/data_norepl/splarseers/output/embed_train.h5', 'embed_train')
ds[0]

array([1.000000e+00, 8.335636e-09, 4.705444e-10], dtype=float32)

In [7]:
with h5py.File('/grid/hackathon/data_norepl/splarseers/output/embed_train.h5', 'r') as f:
    print(f['embed_train'].shape)


(210308, 5000, 3)


In [3]:
with h5py.File('./test.h5', 'w') as f:
    f.create_dataset('data', data=arr)



In [5]:
with HDF3DIterator('./test.h5', 'data') as hdf:
    print(hdf[0])
    print(hdf[0:10])
    print(hdf[0,0])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31]
 [ 32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49
   50  51  52  53  54  55  56  57  58  59  60  61  62  63]
 [ 64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
   82  83  84  85  86  87  88  89  90  91  92  93  94  95]
 [ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
  114 115 116 117 118 119 120 121 122 123 124 125 126 127]
 [128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
  146 147 148 149 150 151 152 153 154 155 156 157 158 159]
 [160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
  178 179 180 181 182 183 184 185 186 187 188 189 190 191]
 [192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
  210 211 212 213 214 215 216

In [6]:
with HDF3DIterator('./test.h5', 'data') as hdf:
    print(hdf[0])
    print(hdf[1])
    print(hdf[2])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63]
[64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87
 88 89 90 91 92 93 94 95]


In [12]:
with HDF3DIterator('./test.h5', 'data') as hdf:
    for i, data in enumerate(hdf):
        print(i, data)
        if i > 10:
            break

0 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
1 [32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63]
2 [64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87
 88 89 90 91 92 93 94 95]
3 [ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
 114 115 116 117 118 119 120 121 122 123 124 125 126 127]
4 [128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159]
5 [160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179 180 181 182 183 184 185 186 187 188 189 190 191]
6 [192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 211 212 213 214 215 216 217 218 219 220 221 222 223]
7 [224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242 243 244 245 246 247 248 249 250 251 252 253 254 255]
8 [256 257 258 259 260 261 262 2

In [16]:
print('jo')

jo


In [17]:
from torch.utils.data import DataLoader

train_ds = HDF3DIterator('./test.h5', 'data')
train_ds.open()

train_dl = DataLoader(train_ds, batch_size=10, num_workers=0)

next(iter(train_dl))




tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31],
        [ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
          46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
          60,  61,  62,  63],
        [ 64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
          78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
          92,  93,  94,  95],
        [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
         110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
         124, 125, 126, 127],
        [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159],
        [160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 1